In [1]:
from scratch.linear_algebra import Vector, dot

def predict(x: Vector, beta: Vector) -> float:
    return dot(x, beta)

In [2]:
from typing import List

def error(x: Vector, y: float, beta: Vector) -> float:
    return predict(x, beta) - y

def squared_error(x: Vector, y: float, beta: Vector) -> float:
    return error(x, y, beta) ** 2

x = [1, 2, 3]
y = 30
beta = [4, 4, 4] # so prediction = 4 + 8 + 12 = 24

assert error(x, y, beta) == -6
assert squared_error(x, y, beta) == 36

In [3]:
def sqerror_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    err = error(x, y, beta)
    return [2 * err * x_i for x_i in x]

assert sqerror_gradient(x, y, beta) == [-12, -24, -36]

In [4]:
import random
import tqdm 
from scratch.linear_algebra import vector_mean
from scratch.gradient_descent import gradient_step

def least_squares_fit(xs: List[Vector], ys: List[float], learning_rate: float = 0.001, num_steps: int = 1000, batch_size: int = 1) -> Vector:
    # Start with a random guess
    guess = [random.random() for _ in xs[0]]
    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess) for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)
    return guess

In [5]:
from pathlib import Path

iris_dataset_file = Path() / 'datasets' / 'iris.dat'
assert iris_dataset_file.is_file(), f'{iris_dataset_file} not found'

In [6]:
import csv

y_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
X, y = [], []
with iris_dataset_file.open('r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row:
            X.append([float(cell) for cell in row[:-1]])
            y.append(y_map[row[-1]])

In [8]:
from scratch.machine_learning import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_pct=0.25)

In [9]:
random.seed(0)
learning_rate = 0.001

beta = least_squares_fit(X_train, y_train, learning_rate, 5000, 25)
beta

least squares fit:   0%|          | 0/5000 [00:00<?, ?it/s]

least squares fit: 100%|██████████| 5000/5000 [00:00<00:00, 8810.50it/s]


[-0.13809606131131974,
 0.031854466650270724,
 0.2946154942174369,
 0.496210760309045]

In [11]:
from scratch.simple_linear_regression import total_sum_of_squares

def multiple_r_squared(xs: List[Vector], ys: Vector, beta: Vector) -> float:
    sum_of_squared_errors = sum(error(x, y, beta) ** 2 for x, y in zip(xs, ys))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(ys)

In [12]:
multiple_r_squared(X_train, y_train, beta)

0.9427479536784517